In [ ]:
# FIRST PRESS PLAY ON THIS CODE BLOCK, TO BE ABLE TO RUN THE FOLLOWING CODE BLOCKS

from math import e
import random
import time

import numpy
import ipywidgets as widgets
from IPython.display import HTML, display

class FlashCardGame():
  def __init__(self, option="aspirated", same_place=True, debug=False):
    self.debug = debug
    self.same_place = same_place

    options = ["plain", "aspirated", "voiced", "nasal", "voiced stop", "affricate"]
    inv_options = {"unaspirated": 1, "voiceless": 2, "obstruent": 3}
    if option in options:
      self.inverse = False
      self.test = options.index(option)
    elif option in inv_options:
      self.inverse = True
      self.test = inv_options[option]
    else:
      raise ValueError(f"Option {option} not recognised")
    self.option = option

    # affricates can only occur in postalveolar place
    if option == "affricate":
      self.same_place = False

    if self.debug: print(self.test)

    self.play()

  def get_choices(self):
    K = numpy.zeros((25, 4), dtype=int)
    Ki = 0
    r = 0
    for i in range(25):
      cp = 0x1780 + i

      pl = 3 if r == 2 else r

      if i % 5 in [1, 3]:
        h = 1
      elif i % 5 == 4 or (i % 5 in [0, 2] and r == 2) or (i % 5 == 0 and r == 4):
        h = 2
      else:
        h = 0

      if i % 5 == 4:
        m = 2
      elif r == 1:
        m = 1
      else:
        m = 0

      K[Ki] = (cp, pl, h, m)
      Ki += 1

      if i % 5 == 4:
        r += 1

    match self.test:
      case 5:
        K1 = K[K[:, 3] != 1]
        K2 = K[K[:, 3] == 1]
      case 4:
        K1 = K[(K[:, 2] != 2) | (K[:, 3] != 0)]
        K2 = K[(K[:, 2] == 2) & (K[:, 3] == 0)]
      case 3:
        K1 = K[K[:, 3] != 2]
        K2 = K[K[:, 3] == 2]
      case _:
        K1 = K[K[:, 2] != self.test]
        K2 = K[K[:, 2] == self.test]

    k2 = random.choice(K2)
    if self.debug: print(f"k2: {chr(k2[0])}")

    if self.same_place:
      alpha_pl = K1[K1[:, 1] == k2[1]]
      if self.debug: print(f"alpha_pl: {', '.join(chr(c) for c in alpha_pl[:,0])}")
      k1 = random.choice(alpha_pl)
    else:
      if self.debug: print(f"K1 cands: {', '.join(chr(c) for c in K1[:,0])}")
      k1 = random.choice(K1)

    if self.debug: print(f"k1: {chr(k1[0])}")
    return [k1[0], k2[0]] if not self.inverse else [k2[0], k1[0]]

  def play(self):
    # Question
    if self.option == "plain":
      sound_name = "a plain (voiceless unaspirated) sound"
    elif self.option[0] in ["a", "e", "i", "o", "u"]:
      sound_name = f"an {self.option} sound"
    else:
      sound_name = f"a {self.option} sound"

    q = widgets.HTML(f"<h1>Which of these Khmer letters represents {sound_name}?</h1>")

    # Empty mark box
    mark = widgets.HTML()

    a = self.get_choices()
    f = random.randint(0,1)

    # set up buttons
    b_layout = widgets.Layout(width="120px", height="80px")
    b1 = widgets.Button(description=chr(a[f]), layout=b_layout)
    b2 = widgets.Button(description=chr(a[(f+1)%2]), layout=b_layout)

    # update CSS for buttons so that inner text of buttons appears large
    display(HTML("""
    <style>
    button {
        font-size: 40px !important;
        color: white !important;
    }
    </style>
    """))

    # add click functionality to buttons
    results = (("Correct!", "green"), ("Incorrect!", "red"))
    def submit(b):
      res = results[(b.id+f)%2]
      if b.responded is False:
        mark.value = f"<h2 style='color:{res[1]};'> {res[0]} </h2>"

        # freeze these buttons
        b1.responded = True
        b2.responded = True

        # change question in 5 seconds
        time.sleep(2)
        self.play()

    b1.on_click(submit)
    b1.id = 1
    b1.responded = False
    b2.on_click(submit)
    b2.id = 2
    b2.responded = False

    # set the buttons to a random colour
    colours = ["purple", "blue", "red", "green", "brown", "gray", "magenta", "orange", "teal", "black", "navy"]
    colour1 = random.choice(colours)
    b1.style.button_color = colour1
    colours.remove(colour1) # remove existing colour so the buttons don't have the same colours
    b2.style.button_color = random.choice(colours)

    # layout
    choices = widgets.HBox([b1, b2], layout=widgets.Layout(justify_content="center"))
    box = widgets.VBox([q, choices, mark], layout=widgets.Layout(align_items="center"))
    display(box)

In [ ]:
FlashCardGame(option = "nasal")

In [ ]:
FlashCardGame(option = "aspirated")

In [ ]:
FlashCardGame(option = "voiced")

In [ ]:
FlashCardGame(option = "plain")

In [ ]:
FlashCardGame(option = "voiceless")

In [ ]:
FlashCardGame(option = "nasal", same_place = False)

In [ ]:
FlashCardGame(option = "aspirated", same_place = False)

In [ ]:
FlashCardGame(option = "voiced", same_place = False)

In [ ]:
FlashCardGame(option = "plain", same_place = False)

In [ ]:
FlashCardGame(option = "voiceless", same_place = False)

In [ ]:
FlashCardGame(option = "affricate")